In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [2]:
import itertools
from operator import itemgetter
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from cvxopt import matrix, solvers, spmatrix

# Import linear demand curve coefficients and initialized paratemers 
from initialize import initialize
from genregparam import linparams

np.set_printoptions(precision=2, suppress=True)

In [3]:
# Parameters for simulation study
n_class = 2
los = 3
capacity = 100
intensity = 1.5
slopes_init = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

# For adaptive pricing algorithm, we have two parameters for step size: param1 and param2
param1, param2 = 200, 10

# Total combinations of arrivals
combs = n_class * 7 * los

In [4]:
# Partitioned protection levels, nested protection levels, representative revenue, and
# discount ration for each virtual bucket, each stay night
buckets, thetas_prt, thetas, rates_vir, ratios = initialize(capacity, intensity, rates_init)
slopes, intercepts = linparams(capacity, intensity, slopes_init, rates_init)

In [5]:
thetas_old = [np.minimum(thetas[i], capacity)[:-1] for i in range(7)]
thetas_old

[array([ 12.,  36., 100.]),
 array([12., 36., 72., 96.]),
 array([12., 24., 72., 84.]),
 array([36., 60., 72.]),
 array([24., 60., 72.]),
 array([12., 24., 36., 60.]),
 array([12., 24., 48., 72.])]

In [6]:
# List all the buckets that a booking class belongs to
# These buckets could span multiple stay nights
bkClass_bkt = []
for i in range(len(buckets)):
    for j in range(len(buckets[i])):
        for item in buckets[i][j]:
            bkClass_bkt.append((item, (i, j)))
            
# Define a function to calculate rates for multiple stay nights
def moving_sum(a, n):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n-1:] 

In [65]:
# In current week, store rates for current week and next week
wk1_rates_AP = rates_init
wk2_rates_AP = rates_init
rates_for_two_weeks_AP = np.concatenate((wk1_rates_AP, wk2_rates_AP)).reshape(2, n_class, 7)
# Derive rates for rate class of r, arrival day of d and length of stay of d
rates_rad_old_AP = [[moving_sum(rates_for_two_weeks_AP[:, i].reshape(2*7), j)[:7] 
  for j in range(1, los+1)] 
 for i in range(n_class)]
# Convert to the right shape for later calculation
rates_rad_old_AP = np.array(rates_rad_old_AP)
rates_rad_old_AP = np.swapaxes(rates_rad_old_AP, 1, 2)

# In current week, store rates for current week and next week
wk1_rates_FCFS = rates_init
wk2_rates_FCFS = rates_init
rates_for_two_weeks_FCFS = np.concatenate((wk1_rates_FCFS, wk2_rates_FCFS)).reshape(2, n_class, 7)
# Derive rates for rate class of r, arrival day of d and length of stay of d
rates_rad_old_FCFS = [[moving_sum(rates_for_two_weeks_FCFS[:, i].reshape(2*7), j)[:7] 
  for j in range(1, los+1)] 
 for i in range(n_class)]
# Convert to the right shape for later calculation
rates_rad_old_FCFS = np.array(rates_rad_old_FCFS)
rates_rad_old_FCFS = np.swapaxes(rates_rad_old_FCFS, 1, 2)
rates_rad_old_AP, rates_rad_old_FCFS

(array([[[135., 270., 405.],
         [135., 270., 405.],
         [135., 270., 405.],
         [135., 270., 378.],
         [135., 243., 351.],
         [108., 216., 351.],
         [108., 243., 378.]],
 
        [[115., 230., 345.],
         [115., 230., 345.],
         [115., 230., 345.],
         [115., 230., 322.],
         [115., 207., 299.],
         [ 92., 184., 299.],
         [ 92., 207., 322.]]]),
 array([[[135., 270., 405.],
         [135., 270., 405.],
         [135., 270., 405.],
         [135., 270., 378.],
         [135., 243., 351.],
         [108., 216., 351.],
         [108., 243., 378.]],
 
        [[115., 230., 345.],
         [115., 230., 345.],
         [115., 230., 345.],
         [115., 230., 322.],
         [115., 207., 299.],
         [ 92., 184., 299.],
         [ 92., 207., 322.]]]))

In [8]:
# Generate mean demand for poisson arrivals for each rate, arrival day, and los combination
mus = intercepts + slopes * rates_rad_old
mus = np.round(mus, 0)

In [9]:
# Flatten arrays for quadratic programming formulation
slopes_flat = slopes.reshape(n_class * 7 * los)
intercepts_flat = intercepts.reshape(n_class * 7 * los)

In [10]:
# Inequality equations, LHS
# We have total number of 42 decision veriables, corresponding to total number of
# rate class, arrival day of week and los combinations.
# Column indexes 0-20 are associated with decision variables for rate class 1
# Column indexes 21-41 are associated with decision variables for rate class 2
G = np.zeros(7 * los * n_class * 7).reshape(7, n_class*7*los)
# Arrivals that span Sunday stay night for rate class 1
G[0,:(7*los)] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 1
G[1,:(7*los)] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,:(7*los)] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,:(7*los)] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
# Arrivals that span Sunday stay night for rate class 2
G[0,(7*los):] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 2
G[1,(7*los):] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,(7*los):] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,(7*los):] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]

h1 = intercepts_flat * G

In [11]:
# Here, be careful. For the capacity constraints, LHS is sum of demands that span
# a specific stay night in question. But decision variables for quadratic programming
# are rates. 
# demand1 + demand2 + demand3 = intercept1 + slope1 * rate1 + intercept2 + slope2 * rate2 + 
# intercept3 + slope3 * rate3 <= capacity
# --> slope1*rate1 + slope2*rate2 + slope3*rate3 <= capacity - (intercept1+intercept2+intercept3)
h1 = np.sum(h1, axis=1)
# G for capacity constraints, Negative identity matrix for non-negativity
G = slopes_flat * G
G = np.concatenate((G, -np.identity(combs)), axis=0)
# Inequality equations, RHS
# First h for capacity rhs and second component for non-negativity rhs.
h = np.concatenate((capacity * np.ones(7) - h1, np.zeros(combs)), axis=0)

In [12]:
# This part is a little bit tedious, but I couldn't find an elegant way of doing it
# Purpose of this section is to make sure optimal rates for e.g., Monday arrival two-night 
# stay is equal to optimal rate for Monday arrival one-night stay and Tuesday arrival one-
# night stay. This is how the rates are calculated in hotel industry for multiple nights stay.
# It is different from airline pricing with multiple legs.
arr1 = [3*i for i in range(14)]
arr2 = [(3*i) for i in range(1, 7)] + [0] + [(3*i) % 42 for i in range(8, 14)] + [21]
arr3 = [3*(i+2) % 21 for i in range(7)] + [3*(i+2) for i in range(7, 12)] + [21] + [24]
arr4 = np.concatenate((np.arange(1, 41, 3), np.arange(2, 42, 3))).tolist()
els = np.concatenate((np.repeat(1.0, 70), np.repeat(-1, 28)))
A = spmatrix(els, np.concatenate((range(28), range(28), range(14, 28), range(28))).tolist(), 
             arr1 + arr1 + arr2 + arr2 + arr3 + arr4)
b = matrix(np.zeros(28))

In [13]:
# Quadratic programming
#                  minimize (1/2)x_T*Q*x + p_T*x
#                  subject to G*x <= h
#                             A*x = b
slopes_diag = np.diag(slopes_flat)
Q = 2 * matrix(-slopes_diag)
p = matrix(-intercepts_flat)

# Convert numpy arrays to cvxopt matrix forms
G = matrix(G)
h = matrix(h)

In [14]:
# Solve quadratic programming
sol = solvers.qp(Q, p, G, h, A, b)
print(sol['status'])

optimal


In [15]:
rates_DP = np.array(sol['x']).reshape(n_class, 7, los)
rates_DP = np.round(rates_DP, 0)

In [16]:
# Check if the rates for multiple los is satisfied, one can look at rate for e.g., Sunday arrival
# three night stay for rate class 1, $405, should equal to Sunday arrival one-night stay,
# Monday arrival one-night stay, and Tuesday arrival, one-night stay.
# And indeed 405 = 112 + 149 + 143 (exits rounding error, of course)
rates_DP

array([[[ 91., 260., 403.],
        [168., 312., 472.],
        [143., 304., 449.],
        [161., 305., 433.],
        [145., 272., 419.],
        [127., 274., 366.],
        [147., 238., 407.]],

       [[ 82., 231., 360.],
        [149., 278., 414.],
        [129., 266., 400.],
        [137., 271., 377.],
        [134., 240., 373.],
        [106., 238., 320.],
        [132., 215., 363.]]])

In [ ]:
######################## Start Simulation ########################
################################################################################################
################################################################################################
################################################################################################
################################################################################################
################################################################################################

In [17]:
# Initialize rates for adaptive pricing algorithm and first-come, first-serve approach
rates_AP = rates_rad_old_AP
rates_FCFS = rates_rad_old_FCFS
# Order of rates as dynamic pricing rates, adaptive pricing rates, and FCFS rates
ratesAll = [rates_DP, rates_AP, rates_FCFS]

# Generate nonhomogenous Poisson process true bookings (i.e., customer demand) on a weekly basis
# the mean of the Poisson process will be equal to the linear demand curve function of the room rates
mus = [np.maximum(intercepts + slopes * rates, 0) for rates in ratesAll]

demands = [[poisson.rvs(mu, size=1) for mu in np.nditer(mus_each)] for mus_each in mus]
demands = [np.array(x).reshape(n_class, 7, los) for x in demands]

demands_next = [[poisson.rvs(mu, size=1) for mu in np.nditer(mus_each)] for mus_each in mus]
demands_next = [np.array(x).reshape(n_class, 7, los) for x in demands_next]

In [18]:
# Consider first week stay nights, our week starts on Sunday, not Monday
nightlyRev = []
wk1SellInfo = []
wk1RoomSold_AP = []
for i in range(7):
    # Sunday of first week, there is no previous week Saturday or Friday arrival that span Sunday
    if i == 0:
        buckets_night = [[rad for rad in rads if rad[1] == 0] for rads in buckets[i]]
    # Monday of first week, there is no previous week Saturday arrival that span Monday
    elif i == 1:
        buckets_night = [[rad for rad in rads if rad[1] != 6] for rads in buckets[i]]
    else:
        buckets_night = buckets[i]
    # Three different algorithms, three different scenarios to consider
    capacity_left = [capacity] * 3
    # Info about the booking class type, rooms sold and revenues (single night revenue)
    soldAll = []
    roomSold_AP = []
    # Reverse the buckets for each night under the assumption that low ranked booking
    # classes arrive first
    pl_idx = 0
    for rads in reversed(buckets_night):
        sold_bucket = []
        sold_bucket_AP = 0
        for rad in rads:
            # For stay night revenue calculation, we only use one-night stay revenue, not
            # multiple stay night total revenue
            rad_single = (rad[0], i, 0)
            # Rooms sold equals smaller of demand and capacity left
            # Dynamic pricing
            sold_DP = min(demands[0][rad], capacity_left[0])
            # Adaptive pricing, the selling amount is constrained by the protection levels for higher classes
            try:
                BL_AP = max(capacity_left[1]-list(reversed(thetas_old[i]))[pl_idx], 0)
            except IndexError:
                BL_AP = capacity_left[1]
            
            sold_AP = min(demands[1][rad], BL_AP)
            sold_bucket_AP += sold_AP
            # First-come, first-serve
            sold_FCFS = min(demands[2][rad], capacity_left[2])
            
            sold = [sold_DP, sold_AP, sold_FCFS]
            
            rev = [ratesAll[i][rad_single] * sold[i] for i in range(len(demands))]
            sold_bucket.append((rad, sold, rev))
            # Update remaining capacity for the next virtual class
            capacity_left = [capacity_left[i]-sold[i] for i in range(len(sold))]
        soldAll.append(sold_bucket)
        roomSold_AP.append(sold_bucket_AP)
        pl_idx += 1

    # Remove empty lists
    soldAll = list(filter(None, soldAll))
    soldAll = list(itertools.chain.from_iterable(soldAll))
    wk1SellInfo.append(soldAll)
    wk1RoomSold_AP.append(roomSold_AP)

    # Extract revenue information and store it in revenue array
    revenue = [soldAll[i][2] for i in range(len(soldAll))]
    nightlyRev.append(revenue)
    
    # Calculate weekly revenue for each algorithm
nightlyRev = [np.array(x) for x in nightlyRev]
revSum = [np.sum(x, axis=0) for x in nightlyRev]
wk1Rev = np.sum(revSum, axis=0)
wk1Rev

array([84020., 66739., 81220.])

In [19]:
wk1RoomSold_AP = [list(reversed(wk1RoomSold_AP[i])) for i in range(len(wk1RoomSold_AP))]
wk1RoomSold_AP = [np.array(x) for x in wk1RoomSold_AP]

# Update protection levels according to the sales info
roomSold_cumsum = [np.cumsum(x) for x in wk1RoomSold_AP]

In [20]:
# Compute if the demand for a class exceeds its corresponding protection levels
Y = [roomSold_cumsum[i][:-1] >= thetas_old[i] for i in range(7)]
# Implement Equation(2) in vanRyzin-McGill 2000
Z = [np.cumproduct(Y[i]) for i in range(7)]
# Calculate H(theta, x)
H = [ratios[i][1:] - Z[i] for i in range(7)]
thetas_new = [thetas_old[i] - (param1/(param2+1)) * H[i] for i in range(7)]
# Truncate at zero and sort it-- nonnegativity of protection levels 
thetas_new = [np.minimum(np.maximum(thetas_new[i], 0), capacity) for i in range(7)]
thetas_new = [sorted(thetas_new[i]) for i in range(7)]
# Round to integers
thetas_new = [np.round(thetas_new[i], 0) for i in range(7)]
thetas_old, thetas_new

([array([ 12.,  36., 100.]),
  array([12., 36., 72., 96.]),
  array([12., 24., 72., 84.]),
  array([36., 60., 72.]),
  array([24., 60., 72.]),
  array([12., 24., 36., 60.]),
  array([12., 24., 48., 72.])],
 [array([ 0., 26., 90.]),
  array([ 0., 23., 59., 88.]),
  array([ 0., 11., 61., 74.]),
  array([23., 47., 60.]),
  array([11., 47., 60.]),
  array([ 0.,  9., 25., 52.]),
  array([14., 30., 37., 63.])])

In [21]:
# Create a dummy booking class 1 so that we can find the partitioned protection levels
# from nested ones in an easy way
thetas_old_full = [np.concatenate(([0], thetas_old[i], [capacity])) for i in range(7)]
thetas_new_full = [np.concatenate(([0], thetas_new[i], [capacity])) for i in range(7)]

# Calculate partitioned protection level changes for each bucket in each night
thetas_old_full_prt = [np.diff(thetas_old_full[i]) for i in range(7)]
thetas_new_full_prt = [np.diff(thetas_new_full[i]) for i in range(7)]

# Percent change for partitioned protection levels
# When divide by zero, we assume the change is 1, or 100%.
thetas_adj = [np.divide(thetas_new_full_prt[i], thetas_old_full_prt[i], 
                        out=(np.zeros_like(thetas_new_full_prt[i])+2),
                       where=thetas_old_full_prt[i]!=0) - 1 for i in range(7)]

In [22]:
bkClass_bkt_uniq = {}
rates_adj = {}
for x, y in bkClass_bkt:
    if x in bkClass_bkt_uniq:
        bkClass_bkt_uniq[x].append((y))
        rates_adj[x].append((thetas_adj[y[0]][y[1]]))
    else:
        bkClass_bkt_uniq[x] = [(y)]
        rates_adj[x] = [(thetas_adj[y[0]][y[1]])]

In [23]:
# Derive average changes for a booking class (i.e., r, a, d combination)
rates_adj_avg = {}
for k, v in rates_adj.items():
    avg_adj = np.round(np.mean(np.array(v)), 4)
    rates_adj_avg[k] = [(avg_adj)]
    single_rate = np.round((rates_rad_old[k] * (1+rates_adj_avg[k][0])) / (k[2] + 1), 0)
    rates_adj_avg[k].append((single_rate))

In [57]:
rate0_new = []
rate1_new = []
for bkt in buckets:
    # Flatten the bucket elements for each day of the week
    bkt_ls = list(itertools.chain.from_iterable(bkt))
    # For each booking class in the falttened list, extract the rates generated by the algorithm
    myRate0_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 0}
    myRate1_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 1}
    # Store it as an array
    myRate0 = np.array(list(myRate0_dict.values()))
    myRate1 = np.array(list(myRate1_dict.values()))
    # Calculate stay night rates for each rate class for each stay night
    myRate0_avg = np.round(np.mean(myRate0, axis=0)[1], 0)
    myRate1_avg = np.round(np.mean(myRate1, axis=0)[1], 0)
    # Create list for new rates to use for next week
    rate0_new.append(myRate0_avg)
    rate1_new.append(myRate1_avg)

# New single night rates for next week
rates_new_AP = np.concatenate((rate0_new, rate1_new)).reshape(n_class, 7)

In [58]:
rates_new_AP

array([[ 90., 160., 126., 124., 124., 114.,  98.],
       [125., 198., 191., 176., 152., 128., 130.]])

In [73]:
buckets

[[[(0, 6, 2)],
  [(1, 6, 2), (0, 0, 2)],
  [(0, 0, 1),
   (0, 5, 2),
   (0, 0, 0),
   (0, 6, 1),
   (1, 0, 0),
   (1, 0, 1),
   (1, 0, 2)],
  [(1, 6, 1), (1, 5, 2)]],
 [[(0, 6, 2)],
  [(1, 6, 2), (0, 0, 2)],
  [(0, 0, 1), (1, 0, 1), (1, 0, 2)],
  [(0, 1, 2), (0, 1, 1)],
  [(0, 1, 0), (1, 1, 0), (1, 1, 1), (1, 1, 2)]],
 [[(0, 0, 2)],
  [(1, 0, 2)],
  [(0, 2, 2), (0, 1, 2), (0, 2, 1), (0, 1, 1)],
  [(0, 2, 0)],
  [(1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 1, 1), (1, 1, 2)]],
 [[(0, 2, 2), (0, 1, 2), (0, 3, 2)],
  [(0, 3, 1), (0, 2, 1)],
  [(0, 3, 0)],
  [(1, 3, 0), (1, 3, 1), (1, 3, 2), (1, 2, 1), (1, 2, 2), (1, 1, 2)]],
 [[(0, 2, 2), (0, 3, 2)],
  [(0, 3, 1), (0, 4, 1), (0, 4, 2)],
  [(0, 4, 0)],
  [(1, 4, 0), (1, 4, 1), (1, 4, 2), (1, 3, 1), (1, 3, 2), (1, 2, 2)]],
 [[(0, 5, 2)],
  [(1, 5, 2)],
  [(0, 3, 2)],
  [(0, 4, 1), (0, 4, 2)],
  [(0, 5, 0),
   (0, 5, 1),
   (1, 5, 0),
   (1, 5, 1),
   (1, 4, 1),
   (1, 4, 2),
   (1, 3, 2)]],
 [[(0, 6, 2)],
  [(1, 6, 2)],
  [(0, 5, 2), (0, 6, 1)],
  

In [72]:
demands_next[2]

array([[[15, 13,  7],
        [28,  7,  9],
        [31, 14,  6],
        [36, 17, 13],
        [25, 12, 12],
        [29,  8,  5],
        [20, 10, 10]],

       [[32,  9,  6],
        [32, 12, 10],
        [15, 10, 14],
        [25, 14,  9],
        [27, 16,  8],
        [26, 14, 12],
        [30, 10, 14]]])

In [62]:
# Next, we consider next week or week 2.
# In current week, store rates for current week and next week
wk1_rates_AP = wk2_rates_AP
wk2_rates_AP = rates_new_AP
rates_for_two_weeks_AP = np.concatenate((wk1_rates_AP, wk2_rates_AP)).reshape(2, n_class, 7)
# Derive rates for rate class of r, arrival day of d and length of stay of d
rates_rad_old_AP = [[moving_sum(rates_for_two_weeks_AP[:, i].reshape(2*7), j)[:7] 
  for j in range(1, los+1)] 
 for i in range(n_class)]
# Convert to the right shape for later calculation
rates_rad_old_AP = np.array(rates_rad_old_AP)
rates_rad_old_AP = np.swapaxes(rates_rad_old_AP, 1, 2)

# In current week, store rates for current week and next week
wk1_rates_AP = wk2_rates_AP
wk2_rates_AP = rates_new_AP
rates_for_two_weeks_AP = np.concatenate((wk1_rates_AP, wk2_rates_AP)).reshape(2, n_class, 7)
# Derive rates for rate class of r, arrival day of d and length of stay of d
rates_rad_old_AP = [[moving_sum(rates_for_two_weeks_AP[:, i].reshape(2*7), j)[:7] 
  for j in range(1, los+1)] 
 for i in range(n_class)]
# Convert to the right shape for later calculation
rates_rad_old_AP = np.array(rates_rad_old_AP)
rates_rad_old_AP = np.swapaxes(rates_rad_old_AP, 1, 2)

array([[[135., 270., 405.],
        [135., 270., 405.],
        [135., 270., 405.],
        [135., 270., 378.],
        [135., 243., 351.],
        [108., 216., 306.],
        [108., 198., 358.]],

       [[115., 230., 345.],
        [115., 230., 345.],
        [115., 230., 345.],
        [115., 230., 322.],
        [115., 207., 299.],
        [ 92., 184., 309.],
        [ 92., 217., 415.]]])

In [53]:
# Consider second week stay nights, our week starts on Sunday, not Monday
nightlyRev_next = []
wk2SellInfo = []
wk2RoomSold_AP = []
for i in range(7):
    buckets_night = buckets[i]
    # Three different algorithms, three different scenarios to consider
    capacity_left = [capacity] * 3
    # Info about the booking class type, rooms sold and revenues (single night revenue)
    soldAll = []
    roomSold_AP = []
    # Reverse the buckets for each night under the assumption that low ranked booking
    # classes arrive first
    pl_idx = 0
    for rads in reversed(buckets_night):
        sold_bucket = []
        sold_bucket_AP = 0
        for rad in rads:
            # For stay night revenue calculation, we only use one-night stay revenue, not
            # multiple stay night total revenue
            rad_single = (rad[0], i, 0)
            # Rooms sold equals smaller of demand and capacity left
            # Dynamic pricing
            sold_DP = min(demands_next[0][rad], capacity_left[0])
            # Adaptive pricing, the selling amount is constrained by the protection levels for higher classes
            try:
                BL_AP = max(capacity_left[1]-list(reversed(thetas_old[i]))[pl_idx], 0)
            except IndexError:
                BL_AP = capacity_left[1]
            
            sold_AP = min(demands_next[1][rad], BL_AP)
            sold_bucket_AP += sold_AP
            # First-come, first-serve
            sold_FCFS = min(demands_next[2][rad], capacity_left[2])
            
            sold = [sold_DP, sold_AP, sold_FCFS]
            
            rev = [ratesAll[i][rad_single] * sold[i] for i in range(len(demands_next))]
            sold_bucket.append((rad, sold, rev))
            # Update remaining capacity for the next virtual class
            capacity_left = [capacity_left[i]-sold[i] for i in range(len(sold))]
        soldAll.append(sold_bucket)
        roomSold_AP.append(sold_bucket_AP)
        pl_idx += 1

    # Remove empty lists
    soldAll = list(filter(None, soldAll))
    soldAll = list(itertools.chain.from_iterable(soldAll))
    wk2SellInfo.append(soldAll)
    wk2RoomSold_AP.append(roomSold_AP)

    # Extract revenue information and store it in revenue array
    revenue = [soldAll[i][2] for i in range(len(soldAll))]
    nightlyRev_next.append(revenue)
    
    # Calculate weekly revenue for each algorithm
nightlyRev_next = [np.array(x) for x in nightlyRev_next]
revSum = [np.sum(x, axis=0) for x in nightlyRev_next]
wk2Rev = np.sum(revSum, axis=0)
wk2Rev

array([92311., 80402., 80292.])